# Exploratory Analysis
<span style="font-size:13pt;">
As a person who plays games, I'm not too familiar with the Metacritic score. I haven't used this to gauge if a game is "great" or not. I hear about games through word of mouth and the constant advertisements that get thrown at me. Metacritic game scores have been around for a while and it may give some insights on how users perceive the games the overall quality of the games. This is just a informal analysis of the metacritic scores attributed to games gathered over time. I'd like to see if I can learn anything about metacritic scores or about game industry. Below is a description of the metacritic score:
Metascore is a weighted average in that we assign more importance, or weight, to some critics and publications than others, based on their quality and overall stature Metascores range from 0-100, with higher scores indicating better overall reviews. We highlight Metascores in three colors so that you can instantly compare: green scores for favorable reviews, yellow scores for mixed reviews, and red scores for unfavorable reviews.(https://www.metacritic.com/about-metascores) Our goal for this analysis is to see if there is any relationship we can between the metascore and other metrics that are provided. 

   

# Reading Data

In [ ]:
import math
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sb

In [ ]:
data = pd.read_csv('../input/metacritic-all-time-games-stats/metacritic_games.csv', parse_dates=['release_date'])
print(data.head(5))

<span style="font-size:13pt;">  Reading in the data went well. However, after checking the output of the data that we are working with I notice the date format is not to my liking. So I chose to update it to a comfortable format. Of course we want to ensure the data we are working with is complete, so we check the number of NULL values, earliest date and latest date we are working with.


In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'], format='%m/%d/%Y')
data['release_year'] = data['release_date'].dt.year
print(data.isnull().sum(), '\n\n')
print('Earliest Date:',data['release_date'].min(),'\nLatest Date: ', data['release_date'].max())

<span style="font-size:13pt;">
It looks like we have quite a lot of NULL values under a few of the columns. However, for this analysis we won’t be using players, rating, and attribute. We can take a quick summary on the numerical values we have in the dataset

In [ ]:
data.describe()

<span style="font-size:13pt;">
The quick descriptive stats above indicate that the mean metascore of all the games in the dataset is ~68.8 with a min value of 8 and a highest score of 99. The median is 71 and upper quantile and lower quantiles are 78 and 61 respectively.
It’s also interesting to note that there are games with no reviews done by critics or users.


# Visualization 
<span style="font-size:13pt;">
While looking at data tables can sometimes be helpful, it doesn't give us the full picture of what is going on. Let’s look at some publisher data

In [ ]:
avg_publisher = data[['publisher','metascore']].groupby(['publisher']).mean('metascore').sort_values(by='metascore', ascending=False).reset_index(drop=False)
print(avg_publisher.head(10))

<span style="font-size:13pt;"> Based on the above output, it appears that top 10 publishers have meta scores between 90 and 94. However, to me this looks unusual since I never heard of these publishers. Let’s look into how many games were from these publishers

In [ ]:
def counts_name_cols(df,colname):
    """
    Function will take in a dataframe and a column name and will get counts based on column name. 
    Then will a add the total counts to the name of the dataset in the following format ""<name> (counts)""
    """
    df['counts'] = ''
    temp_counts = data[[colname,'name']].groupby([colname]).count().reset_index(drop=False)
    for cname in (df[colname].unique()):
        df.loc[df[colname]==cname,'counts'] = temp_counts.loc[temp_counts[colname]==cname,'name'].values
    df['pname'] = df[colname] + "("+ df['counts'].astype(str) + ")"    
    return df   

avg_publisher = counts_name_cols(avg_publisher,'publisher')
print(avg_publisher[['publisher','metascore','counts']].head(10))

<span style="font-size:13pt;"> Looking at the output, it makes sense why these publishers have such a high average metascore. It's because the very few games they had published already all had very high scores. For our purposes, this is not helpful. I want to look at publishers that have published many games. Let’s filter down the list to publishers who have created more than 300 titles.

In [ ]:
avg_publisher = counts_name_cols(avg_publisher,'publisher')
avg_publisher_sub = avg_publisher.loc[avg_publisher['counts']>300].reset_index(drop=True)
ax = avg_publisher_sub.plot.barh(x='pname', y='metascore',xticks=np.arange(0,110,10), xlabel='Publisher(#Titles)'
                                     ,title='Publishers Average Meta Scores', ylabel='Metascore', figsize=(15,5))

<span style="font-size:13pt;"> Great! It appears that the top publishers are some of the ones that I've known. It's interesting to see that EA sports would have the highest metascore. Let’s look into if there is any difference across platforms

In [ ]:
avg_platform = data[['platform','metascore']].groupby(['platform']).mean('metascore').sort_values(by='metascore', ascending=False).reset_index(drop=False)
avg_platform = counts_name_cols(avg_platform,'platform')
print(avg_platform[['platform','metascore','counts']].head(15))

<span style="font-size:13pt;">From the above output, it looks like the average for the platforms are between the ranges of 62-75 which according to Metacritic.com is between "Mixed or Average Review" to "Generally Favorable". However, some of these are unfamiliar, which is why we also added the number of titles made per platform.
Based on the new output, the N64 being the top average metascore seems to be "skewed" in the larger picture since there are platforms that have made thousands of games. Let’s filter our data to only platforms that have over more than 800 games and see how their metascores compare 

In [ ]:
avg_platform_sub = avg_platform.loc[avg_platform['counts']>800].reset_index(drop=True)
ax = avg_platform_sub.plot.barh(x='pname', y='metascore',xticks=np.arange(0,110,5), xlabel='Platform(#Titles)'
                                     ,title='Platform Average Meta Scores', ylabel='Metascore', figsize=(15,5))

<span style="font-size:13pt;">The average metascore ranges appears to be between 63 and 71 which for our case is "Average". Its good to see that the PC platform has is in the top 3 since I only really have played PC games. Not only it is in the top 3, it also has a much larger library of games with over 5000 titles! Let’s look at how PC games from the year 2000 and onward does.

In [ ]:
def top_15_bar(data,plot_variable): 
    """
    Function will take in a dataframe and a variable name to create a barplot
    """
    genre_subdata = subdata.groupby(plot_variable).count().reset_index(drop=False)
    genre_subdata = genre_subdata.sort_values(by='name', ascending=False).head(15)
    genre_subdata_counts = genre_subdata.iloc[:,0:2] 
    genre_subdata_counts.columns = [plot_variable, 'Counts'] 
    plt.bar(genre_subdata_counts[plot_variable], genre_subdata_counts['Counts'])
    plt.xticks(rotation=90); plt.title('Top 15: '+str(plot_variable))
    return 

subdata = data.loc[data['platform'] == 'PC'].reset_index(drop=True) 
subdata = subdata.loc[subdata['release_year'] >= 2000].reset_index(drop=True) 
top_15_bar(subdata,'genre(s)')

top5list_genre = ['Action','Strategy','Role-Playing','Adventure','Action Adventure']
top5genres_data = subdata[subdata['genre(s)'].isin(top5list_genre)].reset_index(drop=True)
print(top5genres_data[['genre(s)','name']].groupby('genre(s)').count())

<span style="font-size:13pt;">
From the above chart, we see that the top PC genres. It was to be expected if we were given the list. From the many games that I have played they all fit those categories. There is a significant drop off after Action adventure games. Let’s drill down into only the top 5 genres. It is important to note that the action genre also has more than double the amount of games than three of the genres. We can look at the distribution of the metascores for each of the genres below

In [ ]:
genre_hist = top5genres_data.hist(column ='metascore',by='genre(s)', figsize=(15,7), bins=10)

<span style="font-size:13pt;">
We can see that data given is distributed nicely. However, the total counts for each genre varies from 100s to 400s which indicates that the Action genre would have the most games 

In [ ]:
top5genres_data.boxplot(column='metascore', by='genre(s)', figsize = (15,10), grid=True, rot=45)
plt.yticks(ticks=np.arange(0,105,5))
plt.ylabel('MetaScores') ;plt.xlabel('Genre(s)');plt.title('')
print("Genre Median:\n ",top5genres_data[['genre(s)','metascore']].groupby('genre(s)').median(),"\n");
print("Genre Mean:\n ",top5genres_data[['genre(s)','metascore']].groupby('genre(s)').mean())

<span style="font-size:13pt;">
Based on the above stats and boxplot, we can see that the median metascore of all five genres which are all close to each other. Based on metacritic they are all mixed review. While this is information is somewhat useful.  We can see there isn't a genre that is more liked among the top five, with only role-playing taking the lead by 3 points. Role-playing also has a higher Q1 and Q3 range among the other four genres. Role-playing game seems to be more favorable compared to the other four. This is also supported by the average of the meta scores based on genres.

In [ ]:
fig, axs = plt.subplots(figsize=(20,5))    
temp = (top5genres_data.groupby(['release_year','genre(s)']).count()).reset_index(drop=False)    
temp = temp[['release_year','genre(s)','name']]

t = sb.lineplot(data=temp, x='release_year',y='name', hue='genre(s)') ; t.set(xlim=(1999,2019),xticks=temp['release_year'].unique())
plt.title('Total Release of Top 5 Genres'); plt.xlabel('Release Year'); plt.ylabel('# of Release')


<span style="font-size:13pt;">As we can see from the above published genres was the highest producing genre over the course of 18 years which is to be expected from our previous histogram. Its important to note that the action genre had a significant jump of PC games release from 2008 to 2013. Based on a quick search what games were release during that time popular titles include [Mass Effect, Fallout, Bioshock, Battlefield, Call of Duty, Red Dead Redemption, Splinter Cell, Assassin’s Creed, etc.] Based on this information, should we expect well known publishing companies to be on the top of the list

In [ ]:
def plot_over_release_year(data,colname,title):
    """
    Function will take in a dataframe, column name and a title to be used. 
    Will plot over time values for column name
    """
    c = i%2 
    r= int(i/2)
    axes[r,c].plot(data['release_year'].astype(int), data[colname])
    axes[r,c].set_title(str(title)) 
    axes[r,c].set_xlabel('Years'); axes[r,c].set_ylabel(colname)
    axes[r,c].set_xticks(data['release_year'])
    return

pubdata = (subdata.groupby('publisher').count())
pubdata = pubdata.sort_values(by='name', ascending=False).head(15).reset_index(drop=False)

publisting = ['Ubisoft','Electronic Arts','Telltale Games','Paradox Interactive','Activision','Atari']
sub_pubdata = pubdata.loc[pubdata['publisher'].isin(publisting),:'name'].reset_index(drop=True)
sub_pubdata.columns = ['publisher','# Games']
print(sub_pubdata)

pub_meta = subdata.loc[subdata['publisher'].isin(publisting)].reset_index(drop=True)
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(25,12))
fig.subplots_adjust(hspace=0.6, wspace=0.4)

for i, p in enumerate(publisting):
    c = i%2 
    r= int(i/2)
    counts_by_pub = pub_meta.loc[pub_meta['publisher']==p].groupby('release_year').mean('metascore').reset_index(drop=False)
    counts_by_pub = counts_by_pub[['release_year','metascore']]
    counts_by_pub.columns = ['release_year', 'metascore'] #rename columns 
    plot_over_release_year(counts_by_pub,'metascore','Publisher: '+p)
    axes[r,c].set_yticks(np.arange(0,100,10))

fig, axs = plt.subplots(figsize=(20,5))    
temp2 = (pub_meta.groupby(['release_year','publisher']).mean('metascore')).reset_index(drop=False)    
temp2 = temp2[['release_year','publisher','metascore']]


total_line = sb.lineplot(data=temp2, x='release_year',y='metascore', hue='publisher') ;total_line.set(xlim=(1999,2019),xticks=temp['release_year'].unique())
plt.title('Average Metascores of Top 5 Publisher'); plt.xlabel('Release Year'); plt.ylabel('Average Metascore');

<span style="font-size:13pt;">Based on the number of total games from each of the publishers listed above, they all seem to have similar average meta scores. Activision, Electronic Arts, and Ubisoft were expected. Telltale Games, Paradox Interactive and Atari were not expected. Overall there isn't much difference overall through time periods except for Activision and Atari in 2012 - 2015. 
A quick google search of Atari's history indicates that in 2013 the company filed for bankruptcy and in 2014 announced a new corporate strategy to shift focus on new audiences.
As for Activision, the google search indicated that they shuttered many of their developers due to losses in revenue from the prior year. From there, Activision put more efforts into their Call of duty franchise


In [ ]:
fig, axes = plt.subplots(figsize=(15,5))
axes.scatter(x='metascore',y='critic_positive', data=subdata, s=100, alpha=0.3, edgecolor = 'white')
axes.set_title('Metascore vs critic_positive'); axes.set_xlabel('Metascores'); axes.set_ylabel('critic_positive');

<span style="font-size:13pt;">As you can see from the above, there is a strong exponential relationship between the metacritic scores and the number of positive critics as the weight of each critic is weighted differently as described in the beginning. This isn't surprising since Metacritic indicated that the score is based on their assignment of importance and weight to of critics’ quality and overall stature.

# Conclusion & Comments
<span style="font-size:13pt;">
Based on the given data already as well as the information on the metacritic scores from metacritic's website we can also agree that the metacritic score is strongly based on critic's review. If we were provided data on the critics that review the games, we can potentially see which critics have a higher weight on the metascores that are assigned. It would have been nice to see the missing years of 2019 and onwards as there have been many new games and consoles that came out. It would have been interesting to see if any of the newer consoles would have made it on the top list or if any of the newer games would have bumped genres higher. A quick search of top PC games in 2019 and 2020 included titles like DOOM, Half-Life, Valorant, Apex Legends, Call of Duty Black Ops and Call of Duty Modern Warfare were so popular at the time everyone I knew was playing those games.

<span style="font-size:13pt;">
During the analysis I notice in the analysis is that the games usually have multiple genres. The dataset only included one genre per game. It'll be interesting to see how the genres were coded if the first genre tagged on a game was chosen. For example, the game "Star Wars Jedi Knight: Dark Forces II" is tagged as an action game while "Star Wars: Knights of the Old Republic" is tagged as a role-playing game. Dark Forces II does also have action-adventure qualities and Knights of the Old Republic does have some action aspects. So having the top genres of "action”, "action-adventure" and "adventure" may very well change if this is clarified. Genre's like "Sci-fi" would also get roped into action or adventure.
 <br>Another thing I notice in the analysis is that PC had an overwhelming number of games in the analysis. If we think about it, PCs themselves don't change as a console. PC's hardware can easily just get upgraded by swapping parts in and out. 
While actual consoles will have to be purchased whole. For example, in the above analysis, if we consider the top metascore average by platform, we can see that there are platforms such as PS2, PS3, PS4, Xbox, Xbox360. If we were to consolidate them into just PlayStation and Xbox entirely, we can get 4,831 and 2,671 games respectively. By doing this the average scores can be shifted and we may find a platform with overwhelming preference.
